# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.112969e+02     1.556956e+00
 * time: 0.09460282325744629
     1     1.031523e+01     1.013987e+00
 * time: 1.0091040134429932
     2    -1.209044e+01     1.075443e+00
 * time: 1.1381418704986572
     3    -3.415203e+01     7.893784e-01
 * time: 1.3004729747772217
     4    -4.760579e+01     5.894924e-01
 * time: 1.4507849216461182
     5    -5.715566e+01     2.036747e-01
 * time: 1.603888988494873
     6    -6.000028e+01     1.156797e-01
 * time: 1.737717866897583
     7    -6.101825e+01     5.613902e-02
 * time: 1.8471088409423828
     8    -6.140289e+01     6.862915e-02
 * time: 1.9637088775634766
     9    -6.168073e+01     2.889919e-02
 * time: 2.0815789699554443
    10    -6.188383e+01     2.307389e-02
 * time: 2.192368984222412
    11    -6.202688e+01     1.799564e-02
 * time: 2.3099489212036133
    12    -6.207925e+01     1.458342e-02
 * time: 2.4185409545898438
    13    -6.213440e+01     1.326578e-02
 * time: 2.535398960113

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671077
    AtomicLocal         -18.8557700
    AtomicNonlocal      14.8522648
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485390 
    Xc                  -19.3336828

    total               -62.261666460377
